In [33]:
import torchvision
import torch
import math
import os
import numpy as np
import pandas as pd
from NNlib import create_dataloaders, get_initial_model, get_test_data, make_itemized_prediction, get_aggregate_prediction

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes = 100
num_test_img = 6667
batch_size = 32
num_workers = os.cpu_count()

NN_train_dir = "data/NN_train_all/"
NN_validate_dir = "data/NN_validate_all/"
NN_test_dir = "data/NN_test_all/"
test_data_dir = "test_filt.csv"
pred_file = 'test_V2.csv'
model_file = 'model_4_3_V6'

In [35]:
data_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

model_0 = get_initial_model(num_classes, device)
model_0.load_state_dict(torch.load(model_file))
train_dataloader, test_dataloader, class_names = create_dataloaders(NN_train_dir,NN_validate_dir,data_transform,batch_size,num_workers)

C:\Users\dgulewic\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dgulewic\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
# import test data
X_test = get_test_data(NN_test_dir, num_test_img, test_data_dir)

In [37]:
# make prediction for each face
test_preds, test_probs = make_itemized_prediction(model_0, X_test, num_test_img)

In [38]:
# create dictionary
indexs = [i for i in range(100)]
prediction_conversion = dict(zip(indexs, class_names))

In [39]:
# make prediction for each picture
filt_pred, dup_number, j_max = get_aggregate_prediction(test_preds, test_probs, test_data_dir)

In [40]:
# convert classes
idxs = np.array(list(set(dup_number))).reshape((j_max,1))
values = [prediction_conversion[key] for key in filt_pred]

# save prediction
prediction = pd.DataFrame(np.column_stack((idxs, values)),columns=['Id', 'Category'])
prediction.to_csv(pred_file, index=False, header=True)